In [6]:
import openai
import pandas as pd
import PyPDF2
from top_secret import OPENAI_API_KEY

# Set the OpenAI API key
openai.api_key = OPENAI_API_KEY

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text.strip()
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""

# Function to load job descriptions from a CSV file
def load_job_details(csv_path):
    try:
        return pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return pd.DataFrame()

# Function to generate the prompt for OpenAI API
def generate_prompt(resume_text, job_description, job_title, job_id):
    return f"""
    # System Role: Job Recommendation Assistant

    Your task is to compare a candidate's resume with a specific job description and determine if the candidate should apply. Only recommend the job if there are **no gaps** between the candidate's skills/experience and the job requirements.

    ## Structure
    - **Job ID**: {job_id}
    - **Job Title**: {job_title}
    - **Job Description**: Provide a concise summary of the job description (2-3 sentences).
    - **Resume Match**:
      - List specific skills and experiences from the resume that align with the job description.
      - Identify **any gaps**, even minor ones, between the resume and the job description.
    - **Recommendation**:
      - Recommend applying for the job **only if no gaps are found**.
      - If gaps exist, explicitly state that the job is not a perfect fit and recommend not applying.

    ## Style
    - Be clear, concise, and specific.
    - Use professional language.

    ## Inputs
    - **Resume**: {resume_text}
    - **Job Description**: {job_description}
    """

# Main script to compare jobs with the resume and generate recommendations
def compare_jobs_and_resume(api_key, resume_path, csv_path):

    # Extract resume text
    resume_text = extract_text_from_pdf(resume_path)
    if not resume_text:
        print("Failed to extract resume text.")
        return

    # Load job details
    job_data = load_job_details(csv_path)
    if job_data.empty:
        print("Failed to load job details.")
        return

    # Iterate through job descriptions and make API calls
    recommendations = []
    for _, job in job_data.iterrows():
        job_id = job.get('job_id', 'Unknown')
        job_title = job.get('title', 'No Title')
        job_description = job.get('description', 'No Description')

        # Generate prompt
        prompt = generate_prompt(resume_text, job_description, job_title, job_id)

        # Make API call
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a Job Recommendation Assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.25  # Lowered for deterministic output
            )
            recommendation = response['choices'][0]['message']['content']

            # Post-process recommendation to filter out jobs with gaps
            if "gap" not in recommendation.lower():  # Only add perfect matches
                recommendations.append((job_id, job_title, recommendation))
            else:
                print(f"Job ID {job_id} has a gap and will not be recommended.")
        except Exception as e:
            print(f"Error making API call for Job ID {job_id}: {e}")

    # Print recommendations
    if recommendations:
        print("Here are the jobs that are a perfect fit for your skills:")
        for job_id, job_title, recommendation in recommendations:
            print(f"---\nJob ID: {job_id}\nJob Title: {job_title}\nRecommendation:\n{recommendation}\n")
    else:
        print("No perfect matches found.")

# Usage example
if __name__ == "__main__":
    # Replace with your OpenAI API key and file paths
    api_key = "your-openai-api-key"  # Replace with your actual OpenAI API key
    resume_path = "saijai_osika_resume.pdf"  # Path to your resume PDF
    csv_path = "job_details.csv"  # Path to job details CSV file

    compare_jobs_and_resume(api_key, resume_path, csv_path)


Job ID 312618060 has a gap and will not be recommended.
Job ID 316163297 has a gap and will not be recommended.
Job ID 317497052 has a gap and will not be recommended.
Job ID 317624589 has a gap and will not be recommended.
Job ID 318606681 has a gap and will not be recommended.
Job ID 319116517 has a gap and will not be recommended.
Job ID 319444211 has a gap and will not be recommended.
Job ID 319660273 has a gap and will not be recommended.
Job ID 320432994 has a gap and will not be recommended.
No perfect matches found.
